# 1

1. List the hyperparameters used in deep learning modeling and describe their roles and impacts. The score will be commensurate with the number of hyperparameters listed and the quality of the descriptions.

2. Implement a deep learning model for the fashion item classification, considering the hyperparameters you mentioned in Question 1. You will use the data set of fashion_mnist, which is available in the tensorflow.keras.datatests. The score will be commensurate with the validation and test accuracies as well as the quality of the modeling.

 - Report a table summary (label, class, frequency, and percent) and a bar graph for the response variable for the training data. Refer to https://www.tensorflow.org/tutorials/keras/classification

 - Use 10000 samples from the training data for validation.

 - Report the contingency table and accuracy for both the validation and test data, along with the code for your best-performing model. Summarize your attempts in terms of hyperparameter adjustments.

# 2 



We want to forecast the sunspot time series data and have considered various methods including DNN, 1D CNN, RNN, Stacked RNN, LSTM, BiLSTM, and Stacked BiLSTM. Although LSTM or BiLSTM are expected to perform better than RNN, the following code demonstrates that RNN achieved the best result. Please improve the results by making appropriate modifications to the LSTM, BiLSTM, and Stacked BiLSTM models.

## Data import and setting training, validation, and test data


In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical

# Load dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly-sunspots.csv'
dataframe = pd.read_csv(url, header=0)

C:\Users\avitr\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
dataframe

,Month,Sunspots
0,1749-01,58.0
1,1749-02,62.6
2,1749-03,70.0
3,1749-04,55.7
4,1749-05,85.0
...,...,...
2815,1983-08,71.8
2816,1983-09,50.3
2817,1983-10,55.8
2818,1983-11,33.3


In [2]:
import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical

# Load dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly-sunspots.csv'
dataframe = pd.read_csv(url, header=0)
sunspots = dataframe['Sunspots'].values

# Normalize the data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
sunspots = scaler.fit_transform(sunspots.reshape(-1, 1)).flatten()

# Prepare data for time series forecasting
def create_dataset(data, look_back=1):
    X, Y = [], []
    for i in range(len(data) - look_back):
        a = data[i:(i + look_back)]
        X.append(a)
        Y.append(data[i + look_back])
    return np.array(X), np.array(Y)

look_back = 12  # Use 12 months of data to predict the next month
X, Y = create_dataset(sunspots, look_back)

# Split into train, validation, and test sets
total_size = len(X)
train_size = int(total_size * 0.6)
validation_size = int(total_size * 0.35)

trainX, trainY = X[:train_size], Y[:train_size]
valX, valY = X[train_size:train_size + validation_size], Y[train_size:train_size + validation_size]
testX, testY = X[train_size + validation_size:], Y[train_size + validation_size:]

print("Training, validation, and test data shapes:")
print(trainX.shape, valX.shape, testX.shape)


Training, validation, and test data shapes:
(1684, 12) (982, 12) (142, 12)


### DNN

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the DNN model
model_dnn = Sequential([
    Flatten(input_shape=(look_back, 1)),  # Flatten the input
    Dense(128, activation='relu'),  # First hidden layer with 128 units and ReLU activation
    Dropout(0.2),  # Dropout for regularization
    Dense(64, activation='relu'),  # Second hidden layer with 64 units and ReLU activation
    Dropout(0.2),  # Additional Dropout for regularization
    Dense(1)  # Output layer with one unit
])

# Compile the model
model_dnn.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Setup checkpoint to save the best model based on validation loss
checkpoint = ModelCheckpoint('best_dnn_model.h5', save_best_only=True, monitor='val_loss', verbose=1)

# Train the model
model_dnn.fit(trainX, trainY, epochs=50, batch_size=32, validation_data=(valX, valY), callbacks=[checkpoint])


########################################
## Model evaluation

from tensorflow.keras.models import load_model

# Load the best saved model
dnn_model = load_model('best_dnn_model.h5')

# Evaluate on test data
dnn_test_mse, dnn_test_mae = dnn_model.evaluate(testX, testY, verbose=0)
print(f'Test MSE: {dnn_test_mse}, Test MAE: {dnn_test_mae}')


Epoch 1/50
53/53 [==============================] - ETA: 0s - loss: 0.0158 - mae: 0.0838
Epoch 1: val_loss improved from inf to 0.00564, saving model to best_dnn_model.h5
53/53 [==============================] - 1s 7ms/step - loss: 0.0158 - mae: 0.0838 - val_loss: 0.0056 - val_mae: 0.0567
Epoch 2/50
46/53 [=========================>....] - ETA: 0s - loss: 0.0058 - mae: 0.0531
Epoch 2: val_loss improved from 0.00564 to 0.00465, saving model to best_dnn_model.h5
53/53 [==============================] - 0s 4ms/step - loss: 0.0060 - mae: 0.0537 - val_loss: 0.0046 - val_mae: 0.0500
Epoch 3/50
52/53 [============================>.] - ETA: 0s - loss: 0.0054 - mae: 0.0513
Epoch 3: val_loss improved from 0.00465 to 0.00423, saving model to best_dnn_model.h5
53/53 [==============================] - 0s 4ms/step - loss: 0.0055 - mae: 0.0514 - val_loss: 0.0042 - val_mae: 0.0479
Epoch 4/50
49/53 [==========================>...] - ETA: 0s - loss: 0.0053 - mae: 0.0502
Epoch 4: val_loss did not improve

### 1D CNN

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the 1D CNN model
model_cnn1d = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(look_back, 1)),  # Convolutional layer
    MaxPooling1D(pool_size=2),  # Pooling layer to reduce dimensionality
    Dropout(0.2),  # Dropout to prevent overfitting
    Conv1D(filters=32, kernel_size=3, activation='relu'),  # Another convolutional layer
    MaxPooling1D(pool_size=2),  # Another pooling layer
    Flatten(),  # Flatten the data for the dense layer
    Dense(50, activation='relu'),  # Dense layer
    Dropout(0.2),  # Another dropout layer for regularization
    Dense(1)  # Output layer
])

# Compile the model
model_cnn1d.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Setup checkpoint to save the best model based on validation loss
checkpoint = ModelCheckpoint('best_cnn1d_model.h5', save_best_only=True, monitor='val_loss', verbose=1)

# Train the model
model_cnn1d.fit(trainX, trainY, epochs=50, batch_size=32, validation_data=(valX, valY), callbacks=[checkpoint])


############################
## Model evaluation

from tensorflow.keras.models import load_model

# Load the best saved model
cnn1d_model = load_model('best_cnn1d_model.h5')

# Evaluate on test data
cnn1d_test_mse, cnn1d_test_mae = cnn1d_model.evaluate(testX, testY, verbose=0)
print(f'Test MSE: {cnn1d_test_mse}, Test MAE: {cnn1d_test_mae}')


Epoch 1/50
36/53 [===================>..........] - ETA: 0s - loss: 0.0135 - mae: 0.0851
Epoch 1: val_loss improved from inf to 0.01049, saving model to best_cnn1d_model.h5
53/53 [==============================] - 1s 8ms/step - loss: 0.0121 - mae: 0.0792 - val_loss: 0.0105 - val_mae: 0.0718
Epoch 2/50
53/53 [==============================] - ETA: 0s - loss: 0.0078 - mae: 0.0627
Epoch 2: val_loss improved from 0.01049 to 0.00952, saving model to best_cnn1d_model.h5
53/53 [==============================] - 0s 5ms/step - loss: 0.0078 - mae: 0.0627 - val_loss: 0.0095 - val_mae: 0.0694
Epoch 3/50
39/53 [=====================>........] - ETA: 0s - loss: 0.0071 - mae: 0.0606
Epoch 3: val_loss improved from 0.00952 to 0.00907, saving model to best_cnn1d_model.h5
53/53 [==============================] - 0s 5ms/step - loss: 0.0073 - mae: 0.0609 - val_loss: 0.0091 - val_mae: 0.0678
Epoch 4/50
52/53 [============================>.] - ETA: 0s - loss: 0.0069 - mae: 0.0596
Epoch 4: val_loss did not i

### RNN

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the Simple RNN model
model_rnn = Sequential([
    SimpleRNN(50, input_shape=(look_back, 1)),  # 50 RNN units
    Dropout(0.2),  # Dropout layer for regularization
    Dense(1)  # Output layer with one unit
])

# Compile the model
model_rnn.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Setup checkpoint to save the best model based on validation loss
checkpoint = ModelCheckpoint('best_rnn_model.h5', save_best_only=True, monitor='val_loss', verbose=1)

# Train the model
model_rnn.fit(trainX, trainY, epochs=50, batch_size=32, validation_data=(valX, valY), callbacks=[checkpoint])

###########################
##  Model evaluation
from tensorflow.keras.models import load_model

# Load the best saved model
rnn_model = load_model('best_rnn_model.h5')

# Evaluate on test data
rnn_test_mse, rnn_test_mae = rnn_model.evaluate(testX, testY, verbose=0)
print(f'Test MSE: {rnn_test_mse}, Test MAE: {rnn_test_mae}')


Epoch 1/50
43/53 [=======================>......] - ETA: 0s - loss: 0.0116 - mae: 0.0756
Epoch 1: val_loss improved from inf to 0.00454, saving model to best_rnn_model.h5
53/53 [==============================] - 2s 10ms/step - loss: 0.0106 - mae: 0.0721 - val_loss: 0.0045 - val_mae: 0.0489
Epoch 2/50
43/53 [=======================>......] - ETA: 0s - loss: 0.0060 - mae: 0.0540
Epoch 2: val_loss improved from 0.00454 to 0.00449, saving model to best_rnn_model.h5
53/53 [==============================] - 0s 6ms/step - loss: 0.0058 - mae: 0.0532 - val_loss: 0.0045 - val_mae: 0.0494
Epoch 3/50
40/53 [=====================>........] - ETA: 0s - loss: 0.0051 - mae: 0.0501
Epoch 3: val_loss improved from 0.00449 to 0.00420, saving model to best_rnn_model.h5
53/53 [==============================] - 0s 6ms/step - loss: 0.0049 - mae: 0.0490 - val_loss: 0.0042 - val_mae: 0.0468
Epoch 4/50
53/53 [==============================] - ETA: 0s - loss: 0.0047 - mae: 0.0484
Epoch 4: val_loss improved from 

### Stacked RNN

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the Stacked RNN model
model_stacked_rnn = Sequential([
    SimpleRNN(50, return_sequences=True, input_shape=(look_back, 1)),  # First RNN layer, returns sequences to the next RNN layer
    Dropout(0.2),  # Dropout for regularization
    SimpleRNN(50),  # Second RNN layer, does not return sequences
    Dropout(0.2),  # Additional Dropout for regularization
    Dense(1)  # Output layer with one unit
])

# Compile the model
model_stacked_rnn.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Setup checkpoint to save the best model based on validation loss
checkpoint = ModelCheckpoint('best_stacked_rnn_model.h5', save_best_only=True, monitor='val_loss', verbose=1)

# Train the model
model_stacked_rnn.fit(trainX, trainY, epochs=50, batch_size=32, validation_data=(valX, valY), callbacks=[checkpoint])


##############################3
## Model evaluation
from tensorflow.keras.models import load_model

# Load the best saved model
stacked_rnn_model = load_model('best_stacked_rnn_model.h5')

# Evaluate on test data
stacked_rnn_test_mse, stacked_rnn_test_mae = stacked_rnn_model.evaluate(testX, testY, verbose=0)
print(f'Test MSE: {stacked_rnn_test_mse}, Test MAE: {stacked_rnn_test_mae}')


Epoch 1/50
51/53 [===========================>..] - ETA: 0s - loss: 0.0745 - mae: 0.1753
Epoch 1: val_loss improved from inf to 0.00769, saving model to best_stacked_rnn_model.h5
53/53 [==============================] - 3s 18ms/step - loss: 0.0727 - mae: 0.1731 - val_loss: 0.0077 - val_mae: 0.0616
Epoch 2/50
50/53 [===========================>..] - ETA: 0s - loss: 0.0160 - mae: 0.0899
Epoch 2: val_loss improved from 0.00769 to 0.00493, saving model to best_stacked_rnn_model.h5
53/53 [==============================] - 1s 11ms/step - loss: 0.0161 - mae: 0.0901 - val_loss: 0.0049 - val_mae: 0.0504
Epoch 3/50
46/53 [=========================>....] - ETA: 0s - loss: 0.0117 - mae: 0.0765
Epoch 3: val_loss improved from 0.00493 to 0.00485, saving model to best_stacked_rnn_model.h5
53/53 [==============================] - 1s 10ms/step - loss: 0.0115 - mae: 0.0760 - val_loss: 0.0049 - val_mae: 0.0505
Epoch 4/50
50/53 [===========================>..] - ETA: 0s - loss: 0.0102 - mae: 0.0698
Epoch 

### LSTM

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint

# Example: LSTM model with dropout
model_lstm = Sequential([
    LSTM(50, input_shape=(look_back, 1)),
    Dropout(0.2),
    Dense(1)
])

model_lstm.compile(optimizer='adam', loss='mse', metrics=['mae'])
checkpoint = ModelCheckpoint('best_lstm_model.h5', save_best_only=True, monitor='val_loss', verbose=1)

model_lstm.fit(trainX, trainY, epochs=50, batch_size=32, validation_data=(valX, valY), callbacks=[checkpoint])


#################################3
## Model evaluation
from tensorflow.keras.models import load_model

# Load the best saved model
lstm_model = load_model('best_lstm_model.h5')

# Evaluate on test data
lstm_test_mse, lstm_test_mae = lstm_model.evaluate(testX, testY, verbose=0)
print(f'Test MSE: {lstm_test_mse}, Test MAE: {lstm_test_mae}')


Epoch 1/50
47/53 [=========================>....] - ETA: 0s - loss: 0.0158 - mae: 0.0891
Epoch 1: val_loss improved from inf to 0.00696, saving model to best_lstm_model.h5
53/53 [==============================] - 4s 20ms/step - loss: 0.0148 - mae: 0.0859 - val_loss: 0.0070 - val_mae: 0.0649
Epoch 2/50
47/53 [=========================>....] - ETA: 0s - loss: 0.0063 - mae: 0.0561
Epoch 2: val_loss improved from 0.00696 to 0.00635, saving model to best_lstm_model.h5
53/53 [==============================] - 1s 9ms/step - loss: 0.0061 - mae: 0.0552 - val_loss: 0.0064 - val_mae: 0.0585
Epoch 3/50
47/53 [=========================>....] - ETA: 0s - loss: 0.0053 - mae: 0.0514
Epoch 3: val_loss improved from 0.00635 to 0.00583, saving model to best_lstm_model.h5
53/53 [==============================] - 1s 10ms/step - loss: 0.0052 - mae: 0.0511 - val_loss: 0.0058 - val_mae: 0.0573
Epoch 4/50
49/53 [==========================>...] - ETA: 0s - loss: 0.0052 - mae: 0.0516
Epoch 4: val_loss improved f

### BiLSTM

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the BiLSTM model
model_bilstm = Sequential([
    Bidirectional(LSTM(50, return_sequences=True), input_shape=(look_back, 1)),  # First BiLSTM layer
    Dropout(0.2),  # Dropout to prevent overfitting
    Bidirectional(LSTM(50)),  # Second BiLSTM layer
    Dropout(0.2),  # Additional dropout for regularization
    Dense(1)  # Output layer
])

# Compile the model
model_bilstm.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Setup checkpoint to save the best model based on validation loss
checkpoint = ModelCheckpoint('best_bilstm_model.h5', save_best_only=True, monitor='val_loss', verbose=1)

# Train the model
model_bilstm.fit(trainX, trainY, epochs=50, batch_size=32, validation_data=(valX, valY), callbacks=[checkpoint])


#######################################3
## Model evaluation

from tensorflow.keras.models import load_model

# Load the best saved model
bilstm_model = load_model('best_bilstm_model.h5')

# Evaluate on test data
bilstm_test_mse, bilstm_test_mae = bilstm_model.evaluate(testX, testY, verbose=0)
print(f'Test MSE: {bilstm_test_mse}, Test MAE: {bilstm_test_mae}')


Epoch 1/50
51/53 [===========================>..] - ETA: 0s - loss: 0.0084 - mae: 0.0658
Epoch 1: val_loss improved from inf to 0.00648, saving model to best_bilstm_model.h5
53/53 [==============================] - 50s 74ms/step - loss: 0.0084 - mae: 0.0656 - val_loss: 0.0065 - val_mae: 0.0598
Epoch 2/50
51/53 [===========================>..] - ETA: 0s - loss: 0.0050 - mae: 0.0508
Epoch 2: val_loss improved from 0.00648 to 0.00549, saving model to best_bilstm_model.h5
53/53 [==============================] - 2s 45ms/step - loss: 0.0050 - mae: 0.0505 - val_loss: 0.0055 - val_mae: 0.0545
Epoch 3/50
52/53 [============================>.] - ETA: 0s - loss: 0.0045 - mae: 0.0481
Epoch 3: val_loss did not improve from 0.00549
53/53 [==============================] - 1s 26ms/step - loss: 0.0046 - mae: 0.0484 - val_loss: 0.0055 - val_mae: 0.0533
Epoch 4/50
53/53 [==============================] - ETA: 0s - loss: 0.0045 - mae: 0.0473
Epoch 4: val_loss improved from 0.00549 to 0.00542, saving mod

### Stacked BiLSTM

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the Stacked BiLSTM model
model_stacked_bilstm = Sequential([
    Bidirectional(LSTM(50, return_sequences=True), input_shape=(look_back, 1)),  # First BiLSTM layer, returns sequences
    Dropout(0.2),  # Dropout for regularization
    Bidirectional(LSTM(30, return_sequences=True)),  # Second BiLSTM layer, also returns sequences
    Dropout(0.2),  # Additional dropout for regularization
    Bidirectional(LSTM(20)),  # Third BiLSTM layer, does not return sequences
    Dropout(0.2),  # Further dropout for regularization
    Dense(1)  # Output layer
])

# Compile the model
model_stacked_bilstm.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Setup checkpoint to save the best model based on validation loss
checkpoint = ModelCheckpoint('best_stacked_bilstm_model.h5', save_best_only=True, monitor='val_loss', verbose=1)

# Train the model
model_stacked_bilstm.fit(trainX, trainY, epochs=50, batch_size=32, validation_data=(valX, valY), callbacks=[checkpoint])


########################################
## Model evaluation

from tensorflow.keras.models import load_model

# Load the best saved model
stacked_bilstm_model = load_model('best_stacked_bilstm_model.h5')

# Evaluate on test data
stacked_bilstm_test_mse, stacked_bilstm_test_mae = stacked_bilstm_model.evaluate(testX, testY, verbose=0)
print(f'Test MSE: {stacked_bilstm_test_mse}, Test MAE: {stacked_bilstm_test_mae}')


Epoch 1/50
53/53 [==============================] - ETA: 0s - loss: 0.0117 - mae: 0.0749
Epoch 1: val_loss improved from inf to 0.00656, saving model to best_stacked_bilstm_model.h5
53/53 [==============================] - 17s 99ms/step - loss: 0.0117 - mae: 0.0749 - val_loss: 0.0066 - val_mae: 0.0608
Epoch 2/50
52/53 [============================>.] - ETA: 0s - loss: 0.0057 - mae: 0.0541
Epoch 2: val_loss improved from 0.00656 to 0.00574, saving model to best_stacked_bilstm_model.h5
53/53 [==============================] - 2s 42ms/step - loss: 0.0057 - mae: 0.0538 - val_loss: 0.0057 - val_mae: 0.0554
Epoch 3/50
53/53 [==============================] - ETA: 0s - loss: 0.0056 - mae: 0.0523
Epoch 3: val_loss did not improve from 0.00574
53/53 [==============================] - 2s 33ms/step - loss: 0.0056 - mae: 0.0523 - val_loss: 0.0059 - val_mae: 0.0550
Epoch 4/50
53/53 [==============================] - ETA: 0s - loss: 0.0052 - mae: 0.0506
Epoch 4: val_loss improved from 0.00574 to 0.0

### Combining all results

In [23]:
import pandas as pd

# Create a list of model names
models = ['DNN', '1D CNN', 'RNN', 'Stacked RNN', 'LSTM', 'BiLSTM', 'Stacked BiLSTM']

# Collect all MSE and MAE results
mse_results = [dnn_test_mse, cnn1d_test_mse, rnn_test_mse, stacked_rnn_test_mse, lstm_test_mse, bilstm_test_mse, stacked_bilstm_test_mse]
mae_results = [dnn_test_mae, cnn1d_test_mae, rnn_test_mae, stacked_rnn_test_mae, lstm_test_mae, bilstm_test_mae, stacked_bilstm_test_mae]

# Create DataFrame for MSE
mse_df = pd.DataFrame({
    'Model': models,
    'Test MSE': mse_results
})

# Create DataFrame for MAE
mae_df = pd.DataFrame({
    'Model': models,
    'Test MAE': mae_results
})

# Display the MSE DataFrame
print("Test MSE for each model:")
print(mse_df)

# Display the MAE DataFrame
print("\nTest MAE for each model:")
print(mae_df)


Test MSE for each model:
            Model  Test MSE
0             DNN  0.005806
1          1D CNN  0.008828
2             RNN  0.005610
3     Stacked RNN  0.005639
4            LSTM  0.005686
5          BiLSTM  0.005653
6  Stacked BiLSTM  0.005885

Test MAE for each model:
            Model  Test MAE
0             DNN  0.058844
1          1D CNN  0.073286
2             RNN  0.056270
3     Stacked RNN  0.056953
4            LSTM  0.057581
5          BiLSTM  0.057728
6  Stacked BiLSTM  0.058594
